Ignore warnings from jupyter notebook

Ignore jupyter notebook warnings

In [1]:
import warnings
warnings.filterwarnings('ignore')

Import libs

In [2]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

Retrieve the data from the wikipedia page by using the BeautifulSoup library

In [3]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
page = requests.get(url)
soup = BeautifulSoup(page.content, "html.parser")

Scrap the info from the page and save it in the content list, composed of a dictionary

In [4]:
table = soup.find("table")
content = []
for row in table.find_all("td"):
    cell = {}
    if row.span.get_text() != 'Not assigned':
        cell['PostalCode'] = row.p.get_text()[:3]
        cell['Borough'] = row.span.get_text().split('(')[0]
        cell['Neighborhood'] = row.span.get_text().split('(')[1].strip(')').replace(' /',',').replace(')',' ').strip(' ')
        content.append(cell)

Save it into a dataframe and make some corrections

In [5]:
df_scrap = pd.DataFrame(content)
df_scrap['Borough'] = df_scrap['Borough'].replace({'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto Stn A',
                                             'East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business',
                                             'EtobicokeNorthwest':'Etobicoke Northwest','East YorkEast Toronto':'East York/East Toronto',
                                             'MississaugaCanada Post Gateway Processing Centre':'Mississauga'})

Show the head of the dataframe

In [6]:
df_scrap.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Queen's Park,Ontario Provincial Government


Show the shape of the Dataframe

In [7]:
df_scrap.shape

(103, 3)

Get the csv file and transform it in a DataFrame

In [8]:
url = "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/labs_v1/Geospatial_Coordinates.csv"
df_coord = pd.read_csv(url)
df_coord.shape

(103, 3)

Show the Dataframe

In [9]:
df_coord.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


Adjust 'PostalCode' column name and Merge DataFrames

In [10]:
df_coord.rename({'Postal Code': 'PostalCode'}, axis='columns', inplace=True)
df_merged = pd.merge(df_scrap, df_coord, on='PostalCode')
df_merged

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Queen's Park,Ontario Provincial Government,43.662301,-79.389494
...,...,...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.653654,-79.506944
99,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
100,M7Y,East Toronto Business,Enclave of M4L,43.662744,-79.321558
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",43.636258,-79.498509
